In [11]:
import geopandas as gpd
import pandas as pd

from shapely.geometry import Point, Polygon
from datetime import datetime


from bokeh.models import Tabs, TabPanel
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS
from bokeh.plotting import show, output_notebook
import bokeh

import acled_conflict_analysis
from acled_conflict_analysis import analysis
from acled_conflict_analysis import visuals

import matplotlib.pyplot as plt
import h3


# Air Pollution near Roads

In [3]:
addis = gpd.read_file('../../data/boundaries/eth_admbnda_adm1_csa_bofedb_2021.shp')
addis = addis[addis['ADM1_EN']=='Addis Ababa']

In [102]:
roads = gpd.read_file('../../data/roads/addis_primary_roads.shp')

In [100]:
boxes_gdf = gpd.read_file('../../data/air_pollution/addis_no2_polygons.shp')

In [139]:
air_pollution = pd.concat([pd.read_csv('../../data/air_pollution/no2_addis_20190101_20191231.csv'),
                            pd.read_csv('../../data/air_pollution/no2_addis_20200101_20201231.csv'),
                            pd.read_csv('../../data/air_pollution/no2_addis_20210101_20211231.csv'),
                            pd.read_csv('../../data/air_pollution/no2_addis_20220101_20221231.csv'),
                            pd.read_csv('../../data/air_pollution/no2_addis_20230101_20231231.csv'),
                            pd.read_csv('../../data/air_pollution/no2_addis_20240101_20241015.csv')

])

In [141]:
air_pollution['date'] = pd.to_datetime(air_pollution['date'])

air_pollution.drop(columns = 'Unnamed: 0', inplace=True)
air_pollution['aoi'] = 'Addis Ababa'
air_pollution.rename(columns = {'date':'event_date'}, inplace=True)

In [16]:
lat_lon = air_pollution[['latitude', 'longitude']].drop_duplicates()
lat_lon = analysis.convert_to_gdf(lat_lon)

In [14]:
def h3_to_polygon(h3_index):
    """Convert H3 index to a Shapely polygon."""
    boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    return Polygon(boundary)

def draw_h3_grid_around_points(gdf, lat_column='latitude', lon_column='longitude', resolution=5):
    """
    Draw H3 grids around specific latitude and longitude in a GeoDataFrame.
    
    Args:
    - gdf: GeoDataFrame containing latitude and longitude points.
    - lat_column: Name of the latitude column in gdf.
    - lon_column: Name of the longitude column in gdf.
    - resolution: H3 resolution (the higher the number, the finer the grid).
    
    Returns:
    - GeoDataFrame with H3 grids.
    """
    h3_indexes = []

    for _, row in gdf.iterrows():
        lat = row[lat_column]
        lon = row[lon_column]
        
        # Get the H3 index for this lat/lon at the specified resolution
        h3_index = h3.geo_to_h3(lat, lon, resolution)
        h3_indexes.append(h3_index)

    # Create polygons for all H3 hexagons
    polygons = [h3_to_polygon(index) for index in h3_indexes]

    # Convert to a new GeoDataFrame
    gdf_hex = gpd.GeoDataFrame(geometry=polygons, crs='EPSG:4326')
    
    return gdf_hex

In [39]:
def gdf_to_h3(gdf, resolution=6):
    """
    Convert polygons in a GeoDataFrame to H3 hexagons at a given resolution.
    
    Args:
    - gdf: Input GeoDataFrame with polygon geometries.
    - resolution: H3 resolution to fill the polygons with hexagons.
    
    Returns:
    - A new GeoDataFrame with H3 hexagon IDs and corresponding hexagon polygons.
    """
    h3_indexes = []
    polygons = []
    
    # Iterate through each polygon in the GeoDataFrame
    for _, row in gdf.iterrows():
        polygon = row.geometry
        
        # Convert Shapely polygon to GeoJSON-like dict format required by h3.polyfill
        geo_json_polygon = {
            'type': 'Polygon',
            'coordinates': [list(polygon.exterior.coords)]
        }
        
        # Convert the coordinates to [lat, lon] format
        geo_json_polygon['coordinates'] = [[[lat, lon] for lon, lat in geo_json_polygon['coordinates'][0]]]
        
        # Get the H3 indexes that cover this polygon
        hexagons = h3.polyfill(geo_json_polygon, resolution)
        
        # Convert H3 indexes to polygons and store
        for h3_index in hexagons:
            h3_indexes.append(h3_index)
            boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
            polygons.append(Polygon(boundary))
    
    # Create a new GeoDataFrame with H3 hex IDs and their corresponding polygons
    gdf_hex = gpd.GeoDataFrame({'h3_index': h3_indexes, 'geometry': polygons}, crs='EPSG:4326')
    
    return gdf_hex

In [105]:
import geopandas as gpd
from shapely.ops import nearest_points

def nearest_road_distance(roads_gdf, boxes_gdf):
    """
    Calculate the distance from each box (polygon) to the nearest road (line) and 
    return a GeoDataFrame with distances.
    
    Args:
    - roads_gdf: GeoDataFrame with road lines (geometry column as LineString).
    - boxes_gdf: GeoDataFrame with box polygons (geometry column as Polygon).
    
    Returns:
    - GeoDataFrame: A new GeoDataFrame containing each box and the distance to the nearest road.
    """
    # Ensure both GeoDataFrames use the same CRS
    if roads_gdf.crs != boxes_gdf.crs:
        roads_gdf = roads_gdf.to_crs(boxes_gdf.crs)
    
    # Create a new column to store the distance to the nearest road
    distances = []
    
    # Iterate over each polygon (box) in boxes_gdf
    for box in boxes_gdf.geometry:
        # Find the nearest road (line) in roads_gdf
        nearest_geom = roads_gdf.geometry.unary_union  # Merge all road lines into one geometry
        nearest_point_on_road = nearest_points(box, nearest_geom)[1]  # Get the point on the road nearest to the box
        
        # Calculate the distance from the box to the nearest road point
        distance = box.distance(nearest_point_on_road)
        distances.append(distance)
    
    # Create a new GeoDataFrame with the original boxes and the calculated distances
    boxes_gdf['distance_to_nearest_road'] = distances
    
    return boxes_gdf


In [114]:
# # Calculate the distances from each box to the nearest road
# roads_utm = roads.to_crs(epsg=32633)
# boxes_utm = boxes_gdf.to_crs(epsg=32633)

# result_gdf = nearest_road_distance(roads_utm, boxes_utm)

result_gdf = result_gdf.to_crs(epsg=4326)

In [164]:
air_pollution_grouped = air_pollution.groupby(['latitude', 'longitude', 'aoi'])['NO2'].mean().reset_index()
distance_to_roads = result_gdf.sjoin(analysis.convert_to_gdf(air_pollution_grouped))

In [170]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
import numpy as np
output_notebook()

source = ColumnDataSource(distance_to_roads[['distance_to_nearest_road', 'NO2']])

x = distance_to_roads['distance_to_nearest_road']
y = distance_to_roads['NO2']

slope, intercept = np.polyfit(x, y, 1)

trend_line = slope * x + intercept

p = figure(title="Distance to nearest primary road vs Average Air Pollution in a location", 
           x_axis_label="Distance to nearest primary road (in m)", y_axis_label="NO2", 
           tools="pan,reset,zoom_in,zoom_out,save")

# Add a scatter plot to the figure
p.scatter('distance_to_nearest_road', 'NO2', size=2, color='blue', source=source, fill_alpha=0.6)

p.line(x, trend_line, color='black', line_width=2)

# Display the plot
show(p)

Loading BokehJS ...

**The correlation coeffecient between the two variables in -0.55. This shows a weak negative correlation.**

In [157]:
air_pollution['weekday'] = air_pollution['event_date'].dt.weekday

def get_weekday_group(weekday):
    if weekday<5:
        return 'weekday'
    else:
        return 'weekend'
    
air_pollution['week_group'] = air_pollution['weekday'].apply(lambda x: get_weekday_group(x))

In [160]:
air_pollution_grouped = air_pollution.groupby(['week_group', 'latitude', 'longitude'])['NO2'].mean().reset_index()
distance_to_roads = result_gdf.sjoin(analysis.convert_to_gdf(air_pollution_grouped))

weekdays = distance_to_roads[distance_to_roads['week_group']=='weekday']
weekends = distance_to_roads[distance_to_roads['week_group']=='weekend']

In [162]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
output_notebook()

source = ColumnDataSource(weekdays[['distance_to_nearest_road', 'NO2']])
p = figure(title="Distance to nearest primary road vs Air Pollution on weekdays", 
           x_axis_label="Distance to nearest primary road (in m)", y_axis_label="NO2", 
           tools="pan,reset,zoom_in,zoom_out,save")

# Add a scatter plot to the figure
p.scatter('distance_to_nearest_road', 'NO2', size=2, color='blue', source=source, fill_alpha=0.6)

# Display the plot
show(p)

Loading BokehJS ...

In [163]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
output_notebook()

source = ColumnDataSource(weekends[['distance_to_nearest_road', 'NO2']])
p = figure(title="Distance to nearest primary road vs Air Pollution on weekends", 
           x_axis_label="Distance to nearest primary road (in m)", y_axis_label="NO2", 
           tools="pan,reset,zoom_in,zoom_out,save")

# Add a scatter plot to the figure
p.scatter('distance_to_nearest_road', 'NO2', size=2, color='blue', source=source, fill_alpha=0.6)

# Display the plot
show(p)

Loading BokehJS ...